In [1]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(instance)

In [2]:
import weakref

In [3]:
class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()

    def __set__(self, instance, value):
        self.values[instance] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(instance)

In [4]:
class Point:
    x = IntegerValue()

In [5]:
p = Point()

In [6]:
print(hex(id(p)))

0x28601678d70


In [7]:
p.x = 100.1

In [8]:
p.x

100

In [9]:
Point.x.values.keyrefs()

[<weakref at 0x00000286016B88B0; to 'Point' at 0x0000028601678D70>]

In [10]:
del p

In [11]:
Point.x.values.keyrefs()

[]

In [12]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(id(instance))

In [13]:
class Point:
    x = IntegerValue()

    def __init__(self, x):
        self.x = x

    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x

In [14]:
p = Point(10.1)

In [15]:
p.x

10

In [16]:
p.x = 20.2

In [17]:
p.x

20

In [19]:
id(p), Point.x.values

(2774572443904, {2774572443904: 20})

In [20]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [21]:
p_id = id(p)

In [22]:
ref_count(p_id)

1

In [23]:
del p

In [24]:
ref_count(p_id)

0

In [25]:
Point.x.values

{2774572443904: 20}

In [26]:
p = Point(10.1)

In [27]:
weak_p = weakref.ref(p)

In [28]:
weak_p

<weakref at 0x000002860164FCE0; to 'Point' at 0x00000286016DCA50>

In [29]:
ref_count(id(p))

1

In [30]:
del p

In [31]:
weak_p

<weakref at 0x000002860164FCE0; dead>

In [32]:
def obj_destroyed(obj):
    print(f'{obj} is being destroyed')

In [33]:
p = Point(10.1)
w = weakref.ref(p, obj_destroyed)

In [34]:
del p

<weakref at 0x000002860176B790; dead> is being destroyed


In [35]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values[id(instance)][1]

    def _remove_object(self, weak_ref):
        print(f'removing dead entry for {weak_ref}')

In [36]:
class Point:
    x = IntegerValue()

In [37]:
p1 = Point()
p2 = Point()

In [38]:
p1.x, p2.x = 10.1, 100.1

In [39]:
p1.x, p2.x

(10, 100)

In [40]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [41]:
del p1

removing dead entry for <weakref at 0x0000028601703650; dead>


In [42]:
class IntegerValue:
    def __init__(self):
        self.values = {}

    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values[id(instance)][1]

    def _remove_object(self, weak_ref):
        reverse_lookup = [key for key, value in self.values.items()
                          if value[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.values[key]

In [43]:
class Point:
    x = IntegerValue()

In [44]:
p = Point()

In [45]:
p.x = 10.1

In [46]:
p.x

10

In [47]:
Point.x.values

{2774572445920: (<weakref at 0x0000028601887FB0; to 'Point' at 0x000002860167B0E0>,
  10)}

In [48]:
del p

In [49]:
Point.x.values

{}

In [50]:
class Person:
    pass

In [51]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__firstlineno__': 1,
              '__static_attributes__': (),
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [52]:
hasattr(Person.__weakref__, '__get__')

True

In [53]:
hasattr(Person.__weakref__, '__set__')

True

In [54]:
p = Person()

In [55]:
hasattr(p, '__weakref__')

True

In [56]:
print(p.__weakref__)

None


In [57]:
w = weakref.ref(p)

In [58]:
print(p.__weakref__)

<weakref at 0x00000286017CDDA0; to 'Person' at 0x000002860167A900>


In [63]:
class Person:
    __slots__ = 'name',

In [64]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__firstlineno__': 1,
              '__slots__': ('name',),
              '__static_attributes__': (),
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

In [65]:
p = Person()

In [66]:
hasattr(p, '__weakref__')

False

In [67]:
w = weakref.ref(p)

TypeError: cannot create weak reference to 'Person' object

In [68]:
class Person:
    __slots__ = 'name', '__weakref__'

In [69]:
p = Person()

In [70]:
weakref.ref(p)

<weakref at 0x0000028601E33380; to 'Person' at 0x0000028601E57410>

In [88]:
class ValidString:
    def __init__(self, min_length=0, max_length=255):
        self.data = {}
        self._min_length = min_length
        self._max_length = max_length

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError('Value must be a string')
        if len(value) < self._min_length:
            raise ValueError(
                f'Value should be at least {self._min_length} characters.'
            )
        if len(value) > self._max_length:
            raise ValueError(
                f'Value cannot exceed {self._max_length} characters.'
            )
        self.data[id(instance)] = (weakref.ref(instance, self._finalize_instance), value)
    
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]
    
    def _finalize_instance(self, weak_ref):
        reverse_lookup = [key for key, value in self.data.items()
                          if value[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.data[key]

In [134]:
class Person:
    __slots__ = '__weakref__',

    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)

    def __eq__(self, other):
        return (
            isinstance(other, Person) and
            self.first_name == other.first_name and
            self.last_name == other.last_name
        )
    
class BankAccount:
    __slots__ = '__weakref__',

    account_number = ValidString(5, 255)

    def __eq__(self, other):
        return isinstance(other, BankAccount) and self.account_number == other.account_number

In [135]:
p1 = Person()

In [136]:
try:
    p1.first_name = ''
except ValueError as ex:
    print(ex)

Value should be at least 1 characters.


In [137]:
p2 = Person()

In [138]:
p1.first_name, p1.last_name = 'Guido', 'van Rossum'
p2.first_name, p2.last_name = 'Raymond', 'Hettinger'

In [139]:
b1,b2 = BankAccount(), BankAccount()

In [140]:
b1.account_number, b2.account_number = 'Savings', 'Checking'

In [141]:
p1.first_name, p1.last_name

('Guido', 'van Rossum')

In [142]:
p2.first_name, p2.last_name

('Raymond', 'Hettinger')

In [143]:
b1.account_number, b2.account_number

('Savings', 'Checking')

In [144]:
Person.first_name.data

{2774573606064: (<weakref at 0x0000028601F71C10; to 'Person' at 0x00000286017964B0>,
  'Guido'),
 2774576198912: (<weakref at 0x0000028601F80F90; to 'Person' at 0x0000028601A0F500>,
  'Raymond')}

In [145]:
Person.last_name.data

{2774573606064: (<weakref at 0x0000028602073C90; to 'Person' at 0x00000286017964B0>,
  'van Rossum'),
 2774576198912: (<weakref at 0x0000028601F814E0; to 'Person' at 0x0000028601A0F500>,
  'Hettinger')}

In [146]:
BankAccount.account_number.data

{2774574782496: (<weakref at 0x0000028601F4AE80; to 'BankAccount' at 0x00000286018B5820>,
  'Savings'),
 2774572633072: (<weakref at 0x0000028602073D30; to 'BankAccount' at 0x00000286016A8BF0>,
  'Checking')}

In [147]:
del p1
del p2
del b1
del b2

In [148]:
Person.first_name.data

{}